In [ ]:
from trimesh import *
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D, art3d
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()
from plotly.subplots import make_subplots
from scipy.spatial import Delaunay
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go

In [ ]:
octahedron = np.array([[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1]])
octFaces = np.array([[1,2,4],[1,3,4],[3,0,4],[0,2,4],[1,3,5],[3,0,5],[0,2,5],[2,1,5]])
oct0=Trimesh(octahedron,octFaces,ALPHA=1/2,BETA=1/8,GAMMA=-1/16)

In [ ]:
oct0.closest_point_naive(np.array([0,1,1]))

In [ ]:
tetrahedron = np.array([[np.sqrt(8/9),0,-1/3],[-np.sqrt(2/9),np.sqrt(2/3),-1/3],[-np.sqrt(2/9),-np.sqrt(2/3),-1/3],[0,0,1]])
tetFaces = np.array([[0,1,2],[0,1,3],[1,2,3],[2,3,0]])
tet0 = Trimesh(tetrahedron,tetFaces)

In [ ]:
oct1 = oct0.subdivide(modified=True)
oct2 = oct1.subdivide(modified=True)
oct3 = oct2.subdivide(modified=True)
#oct4 = oct3.subdivide(modified=True)

In [ ]:
def surfacePlot(matrix,title=None):
    matrix = np.rot90(np.rot90(matrix.T))
    aspect = matrix.shape[1]/matrix.shape[0]
    fig = go.Figure(data=[go.Surface(z=matrix)])
    
    camera = dict(eye=dict(x=0., y=0., z=2.5))
    
    if aspect>=1:
        fig.update_layout(autosize=False,height=800,width=600,
                          title=title,
                          scene_aspectmode='data',
                          margin=dict(l=65, r=50, b=65, t=90),
                          scene_camera=camera)
    else:
        fig.update_layout(autosize=False,height=600,width=800,
                          title=title,
                          scene_aspectmode='data',
                          margin=dict(l=65, r=50, b=65, t=90),
                          scene_camera=camera)
    fig.show()

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(10,10))
for mesh in (oct1,oct2,oct3):
    plane = np.isclose(mesh.vertices[:,2],np.zeros(mesh.vertices[:,2].shape))
    P,Q,A,B = mesh.filters
    azimuth = np.arctan2(mesh.vertices[:,1],mesh.vertices[:,0])[plane].flatten()
    sorts = np.argsort(azimuth)
    axs[0,0].plot(azimuth[sorts],A[0,:][plane].flatten()[sorts],'--o',label=f'Level {mesh.level-1}')
    
    axs[0,1].plot(azimuth[sorts],B[0,:][plane].flatten()[sorts],'--o',label=f'Level {mesh.level-1}')

    axs[1,0].plot(azimuth[sorts],P[:,0][plane].flatten()[sorts],'--o',label=f'Level {mesh.level-1}')
    
    axs[1,1].plot(azimuth[sorts],Q[:,0][plane].flatten()[sorts],'--o',label=f'Level {mesh.level-1}')
    
axs[0,0].set_title('First Row of A')
axs[0,1].set_title('First Row of B')
axs[1,0].set_title('First Column of P')
axs[1,1].set_title('First Column of Q')
axs[0,0].legend()
axs[0,1].legend()
axs[1,0].legend()
axs[1,1].legend()
fig.tight_layout()

In [ ]:
print('BIORTHOGONALITY TESTS \n')
for mesh in (oct1,oct2,oct3):
    P,Q,A,B = mesh.filters
    AB = np.vstack((A,B))
    PQ = np.hstack((P,Q))
    M = AB @ PQ
    N = PQ @ AB
    print(f':::: level {mesh.level} ::::')
    print()
    print('[[A][B]]@[PQ]== 1')
    print(np.allclose(M, np.eye(M.shape[0])))
    print()
    print('[PQ]@[[A][B]]== 1')
    print(np.allclose(N, np.eye(N.shape[0])))
    print()
    print('B @ P == 0')
    BP = B@P
    print(np.allclose(BP, np.zeros(BP.shape)))
    print()
    print('A @ Q == 0')
    AQ = A@Q
    print(np.allclose(AQ, np.zeros(AQ.shape)))
    print()
    print('A @ P == 1')
    AP = A@P
    print(np.allclose(AP, np.eye(AP.shape[0])))
    print()
    print('B @ Q == 1')
    BQ = B@Q
    print(np.allclose(BQ, np.eye(BQ.shape[0])))
    print()
    print('P@A + Q@B == 1')
    print(np.allclose(P@A + Q@B,np.eye((P@A+Q@B).shape[0])))
    print()